In [3]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from torrequest import TorRequest
from pprint import pprint
import pandas as pd
from retrying import retry

In [4]:
sampled_players = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_list_of_sampled_players.csv')

In [8]:
sampled_players.head()

,player,lastname,team,link_tm,link_ws
0,Max Clark,Clark,Vitesse,/max-clark/profil/spieler/183291,/Players/313996/Show/Max-Clark
1,Sergio Postigo,Postigo,Levante,/sergio-postigo/profil/spieler/158791,/Players/109896/Show/Sergio-Postigo
2,Stanislav Iljutcenko,Iljutcenko,Duisburg,/stanislav-iljutcenko/profil/spieler/191292,/Players/132901/Show/Stanislav-Iljutcenko
3,Iván Marcone,Marcone,Boca Juniors,/ivan-marcone/profil/spieler/90451,/Players/125246/Show/Iván-Marcone
4,Florent Mollet,Mollet,Montpellier,/florent-mollet/profil/spieler/222859,/Players/114169/Show/Florent-Mollet


In [22]:
class get_awards():
    
    def _init_(self):
        pass
        
    @retry(wait_fixed=3000)
    def get_soup(self, URL):
        with TorRequest() as tr:
            tr.reset_identity()
            headers = {"User-Agent":'Mozilla/5.0 (X11; Linux x86_64; rv:7.0.1) Gecko/20100101 Firefox/7.7'}
            r = tr.get(URL, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            return soup
        
    def extract_awards(self, soup):
        awards = []
        for each in soup.find_all('tr', attrs={'class':'bg_Sturm'}):
            try:
                awards.append(each.find('td', attrs={'colspan':'3'}).text)
            except:
                awards.append(np.nan)
        return awards
    
    def transform_dataset(self, player, awards):
        awards_dict = {}

        for each in awards: 
            
            # Excluding "participant" awards, excepting for World Cup
            if "participant" not in each.split(" ", maxsplit=1)[1] or "World Cup" in each.split(" ", maxsplit=1)[1]:
                awards_dict[each.split(" ", maxsplit=1)[1]] = each.split(" ", maxsplit=1)[0].replace("x","")

        awards_df = pd.DataFrame(awards_dict,index=[player])
        return awards_df
    
    def get_awards(self, list_of_players):
        self.list_of_players = list_of_players
        awards_df = pd.DataFrame()
        list_players = list_of_players.link_tm

        for each in tqdm_notebook(list_players):
            
            n_link = each.replace('profil','erfolge')
            new_link = 'https://www.transfermarkt.co.uk' + n_link
            soup = self.get_soup(new_link)
            awards = self.extract_awards(soup)
            player_df = self.transform_dataset(each, awards)

            awards_df = pd.concat([awards_df, player_df], axis=0, sort=True)
            print(new_link)

        return awards_df

In [23]:
getting_awards = get_awards()

In [ ]:
test_awards_df = getting_awards.get_awards(sampled_players)

In [33]:
test_awards_df.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_players_awards.csv',index=True)


In [30]:
test_awards_df.shape

(1200, 259)

In [34]:
final_awards = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/final_players_awards.csv')